<a href="https://colab.research.google.com/github/AnnSenina/Python_for_CL/blob/main/homeworks/Hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Работа с файлами, pandas + парсинг
Собираем корпус эко-новостей с сайта новостного издания "Ведомости"

[По этой ссылке можно найти главную страницу](https://www.vedomosti.ru/ecology)

Что нам предстоит сделать:

достать все заголовки новостей в главной страницы + текст каждой новости

сохранить в датафрейм с колонками "ссылка", "дата", "заголовок", "текст"

сохранить датафрейм в файл

положить код и получившийся файл в свой репозиторий на гитхабе

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url = "https://www.vedomosti.ru/ecology"

page = rq.get(url)
page.encoding = "utf-8"

soup = BeautifulSoup(page.text, features = "html.parser")

In [3]:
#print(page)
#print(soup.prettify())
#soup.find_all("a")

In [4]:
main_links = []
link_parts = []
issue_links=[]

for link in soup.find_all("a"):
        main_links.append(link.get("href"))
        
main_links=list(set(main_links))
        
for x in main_links:
    if "/ecology/release" in x:
        link_parts.append(x)

for i in link_parts:
    issue_links.append("https://www.vedomosti.ru"+i)

In [5]:
#print(issue_links)
#print(len(issue_links))

In [6]:
#часть 2

In [7]:
all_sub_links = []
sub_link_parts = []
sub_issue_links=[]

for i in issue_links:
    url2 = i
    page2 = rq.get(url2)
    page2.encoding = "utf-8"
    soup2 = BeautifulSoup(page2.text, features = "html.parser")
    for link in soup2.find_all("a"):
        all_sub_links.append(link.get("href"))

for x in all_sub_links:
    try:
        if "articles" in x or "columns" in x:
            sub_link_parts.append(x)
    except:
        pass

for i in sub_link_parts:
    sub_issue_links.append("https://www.vedomosti.ru"+i)    

In [8]:
#print(sub_issue_links)
#print(len(sub_issue_links))

In [9]:
# Часть 3

In [10]:
names=[]
date_time=[]
titles=[]
subtitles=[]
current_string=[]
current_text=[]

for i in sub_issue_links:
    temp_text=[]
    url3 = i
    page3=rq.get(url3)
    page3.encoding="utf-8"
    soup3=BeautifulSoup(page3.text, features="html.parser")
    for a in soup3.find_all("p")[:-16]:
        temp_text.append(a.text)
    current_string = " ".join(temp_text)
    current_text.append(current_string)    
    date_time.append(soup3.find_all("time")[0].text)        
    titles.append(soup3.find_all("h1")[0].text)
    subtitles.append(soup3.find_all("meta")[-2].get("content"))
    names.append(soup3.find_all("a")[13].text)

In [14]:
#print(date_time)
#print(names)
#print(titles)
#print(subtitles)

In [15]:
# Часть 4

In [16]:
cleared_names=[]

for i in names:
        cn=re.sub(r"(\n\s+)","",i)
        cleared_names.append(cn)
print(cleared_names)

['Марина Кочетова', 'Виктор Данилов-Данильян', 'Маргарита Парфененкова', 'Марина Кочетова', 'Маргарита Парфененкова', 'Маргарита Парфененкова', 'Марина Кочетова', 'Анна Героева', 'Ведомости.Экология ', 'Маргарита Парфененкова', 'Марина Кочетова', 'Маргарита Парфененкова', 'Петр Сапожников', 'Ведомости.Экология ', 'Михаил Крейндлин']


In [17]:
cleared_date_time=[]

for i in date_time:
    cdt=re.sub(r"(\s\/)","",i)
    cleared_date_time.append(cdt)

print(cleared_date_time)

#print(current_text)[:20]

['12 декабря, 10:34', '12 декабря, 09:30', '12 декабря, 10:04', '08 декабря, 12:50', '08 декабря, 12:19', '08 декабря, 11:43', '08 декабря, 11:24', '08 сентября, 11:28', '08 сентября, 11:13', '08 сентября, 10:25', '08 сентября, 09:11', '10 ноября, 11:58', '10 ноября, 11:26', '10 ноября, 11:38', '10 ноября, 11:09']


In [18]:
cleared_titles=[]

for i in titles:
    ct=re.sub(r"(\n\s+)","",i)
    cleared_titles.append(ct)

print(cleared_titles)

['Метан – недолговечный, но коварный', 'Климатические изменения и устойчивое развитие', 'Добывающей отрасли прогнозируют диверсификацию', 'Энергетика с нулевым следом', '«Россия» станет самым мощным ледоколом в мире', 'Альтернатива «большому» атому', 'Атомная экология', 'Весной и летом 2022 г. леса в России горели меньше, чем в среднем в XXI веке', '«Мониторинг Байкала — это не академическая наука, а участь сантехника»', 'Медотходы класса А могут приравнять к офисному и бытовому мусору', 'Благотворительные продукты освободят от НДС', 'Россия теплеет вдвое быстрее, чем остальной мир', 'Запретят ли собирать кедровые орехи', '«В России отсутствует специализированная инфраструктура сбора текстильных отходов»', '«Рубка байкальского леса приведет к непоправимым последствиям для экосистемы озера»']


In [19]:
# Часть 5


In [23]:
data_list=cleared_date_time,cleared_names,cleared_titles,subtitles,current_text

eco_df = pd.DataFrame(data_list, columns=sub_issue_links).T
eco_df=eco_df.rename(columns={0:"Date and time", 1:"Author", 2:"Title", 3:"Subtitle", 4:"Text"})
display(eco_df)

,Date and time,Author,Title,Subtitle,Text
https://www.vedomosti.ru/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii,"12 декабря, 10:34",Марина Кочетова,"Метан – недолговечный, но коварный",Одна из важнейших задач в рамках стратегии уст...,Российские нефтегазовые компании активно участ...
https://www.vedomosti.ru/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-i-ustoichivoe-razvitie,"12 декабря, 09:30",Виктор Данилов-Данильян,Климатические изменения и устойчивое развитие,Современные климатические изменения представля...,Современные климатические изменения представля...
https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/12/954778-dobivayuschei-otrasli-prognoziruyut-diversifikatsiyu,"12 декабря, 10:04",Маргарита Парфененкова,Добывающей отрасли прогнозируют диверсификацию,Дружественные страны планируют наращивать сотр...,В условиях макроэкономической нестабильности и...
https://www.vedomosti.ru/ecology/climate/articles/2022/12/08/954315-energetika-nulevim-sledom,"08 декабря, 12:50",Марина Кочетова,Энергетика с нулевым следом,"Россия, занимая 2-е место среди стран Европы п...","Климатическая повестка, декарбонизация, зелена..."
https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/08/954309-rossiya-stanet-samim-moschnim-ledokolom-v-mire,"08 декабря, 12:19",Маргарита Парфененкова,«Россия» станет самым мощным ледоколом в мире,Леонид Ирлица: «Без атомных ледоколов невозмож...,"И. о. генерального директора ФГУП «Атомфлот», ..."
https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/08/954298-alternativa-bolshomu-atomu,"08 декабря, 11:43",Маргарита Парфененкова,Альтернатива «большому» атому,"АЭС малой мощности, ветроэлектростанции и сист...",Их работа не ограничивается строительством и э...
https://www.vedomosti.ru/ecology/protection_nature/articles/2022/12/08/954286-atomnaya-ekologiya,"08 декабря, 11:24",Марина Кочетова,Атомная экология,Российские ядерщики защищают природу в обоих п...,"Существует множество примеров, показывающих, ч..."
https://www.vedomosti.ru/ecology/protection_nature/articles/2022/09/08/939891-vesnoi-i-letom-2022-g-lesa-v-rossii-goreli-menshe-chem-v-srednem-v-xxi-veke,"08 сентября, 11:28",Анна Героева,Весной и летом 2022 г. леса в России горели ме...,"В этом году огонь прошел 3,3 млн га леса","Пожары в Рязанской области, запах гари в Москв..."
https://www.vedomosti.ru/ecology/protection_nature/articles/2022/09/08/939876-monitoring-baikala-eto-ne-akademicheskaya-nauka-a-uchast-santehnika,"08 сентября, 11:13",Ведомости.Экология,«Мониторинг Байкала — это не академическая нау...,Руководитель научной экспедиции по экологическ...,Руководитель научной экспедиции по экологическ...
https://www.vedomosti.ru/ecology/regulation/articles/2022/09/08/939853-medothodi-klassa-a-mogut-priravnyat-k-ofisnomu-i-bitovomu-musoru,"08 сентября, 10:25",Маргарита Парфененкова,Медотходы класса А могут приравнять к офисному...,Исторически сложившийся барьер между медицинск...,Председатель комитета Совета Федерации по агра...


In [21]:
# Часть 6

In [24]:
eco_df.to_csv('Vedomosti_Eco.csv', encoding='utf-8') 